# Plan de Trabajo


Deberás realizar un análisis exploratorio de datos. Al final de Jupyter Notebook, escribe: Una lista de preguntas aclaratorias.

Un plan aproximado para resolver la tarea, que especifica de 3 a 5 pasos básicos y los explica en uno o dos enunciados El líder del equipo revisará tus preguntas y plan de trabajo.

Las preguntas serán respondidas durante una videollamada. El código será revisado por el líder del equipo solo si hay algunas dudas.

Haz el proyecto en tu ordenador y súbelo cuando hayas terminado. Si tienes algún problema, intenta usar nuestra interfaz.

# Proyecto Final

Al operador de telecomunicaciones Interconnect le gustaría poder pronosticar su tasa de cancelación de clientes. Si se descubre que un usuario o usuaria planea irse, se le ofrecerán códigos promocionales y opciones de planes especiales. El equipo de marketing de Interconnect ha recopilado algunos de los datos personales de sus clientes, incluyendo información sobre sus planes y contratos.

### Servicios de Interconnect

Interconnect proporciona principalmente dos tipos de servicios:

1. Comunicación por teléfono fijo. El teléfono se puede conectar a varias líneas de manera simultánea.
2. Internet. La red se puede configurar a través de una línea telefónica (DSL, *línea de abonado digital*) o a través de un cable de fibra óptica.

Algunos otros servicios que ofrece la empresa incluyen:

- Seguridad en Internet: software antivirus (*ProtecciónDeDispositivo*) y un bloqueador de sitios web maliciosos (*SeguridadEnLínea*).
- Una línea de soporte técnico (*SoporteTécnico*).
- Almacenamiento de archivos en la nube y backup de datos (*BackupOnline*).
- Streaming de TV (*StreamingTV*) y directorio de películas (*StreamingPelículas*)

La clientela puede elegir entre un pago mensual o firmar un contrato de 1 o 2 años. Puede utilizar varios métodos de pago y recibir una factura electrónica después de una transacción.

### Descripción de los datos

Los datos consisten en archivos obtenidos de diferentes fuentes:

- `contract.csv` — información del contrato;
- `personal.csv` — datos personales del cliente;
- `internet.csv` — información sobre los servicios de Internet;
- `phone.csv` — información sobre los servicios telefónicos.

En cada archivo, la columna `customerID` (ID de cliente) contiene un código único asignado a cada cliente. La información del contrato es válida a partir del 1 de febrero de 2020.

## 1. Preparación de los Datos

In [6]:
# INSTALAMOS LAS LIBRERIAS

!pip install sidetable

In [7]:
# CARGAMOS LAS LIBRERIAS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sidetable as stb
from functools import reduce

In [30]:
# CARGAMOS LA DATA

try:
    contract = pd.read_csv('e:\\Users\\Admin\\Desktop\\Python Projects\\contract.csv')
    internet = pd.read_csv('e:\\Users\\Admin\\Desktop\\Python Projects\\internet.csv')
    personal = pd.read_csv('e:\\Users\\Admin\\Desktop\\Python Projects\\personal.csv')
    phone = pd.read_csv('e:\\Users\\Admin\\Desktop\\Python Projects\\phone.csv')

except: 
    contract = pd.read_csv('/datasets/final_provider/contract.csv')
    internet = pd.read_csv('/datasets/final_provider/internet.csv')
    personal = pd.read_csv('/datasets/final_provider/personal.csv')
    phone = pd.read_csv('/datasets/final_provider/phone.csv')


### 1.1 Información General de 'contract'

In [31]:
# MOSTRAMOS UNA VISTA PREVIA

contract.sample(5)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
3306,9643-AVVWI,2019-11-01,No,Month-to-month,Yes,Electronic check,80.00,241.3
6161,4343-EJVQB,2019-07-01,No,Month-to-month,No,Mailed check,74.35,533.6
5953,6888-SBYAI,2020-01-01,No,Month-to-month,Yes,Mailed check,50.70,50.7
5505,4776-XSKYQ,2019-02-01,No,One year,No,Credit card (automatic),30.25,368.85
2898,6505-OZNPG,2019-08-01,No,Two year,Yes,Mailed check,63.40,348.8


In [32]:
# MODIFICAMOS LOS NOMBRES DE LAS COLUMNAS

contract_nombres = {
    'customerID': 'customer_id',
    'BeginDate': 'begin_date',
    'EndDate': 'end_date',
    'Type': 'type',
    'PaperlessBilling': 'paperless_billing',
    'PaymentMethod': 'payment_method',
    'MonthlyCharges': 'monthly_charges',
    'TotalCharges': 'total_charges'
}

contract.rename(columns = contract_nombres, inplace = True)

contract.head()

,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65


Tenemos la siguiente información:
    
* **customer_id**: código único para cada cliente.

* **begin_date**: fecha en donde se suscribe el cliente.
    
* **end_date**: fecha en donde finaliza la suscripción el cliente.
    
* **type**: tipo de contrato, mensual o anual.
    
* **paperless_billing**: tipo de envío de factura, electrónica o por correo.
    
* **monthly_charges**: precio de la cuota mensual en dólares.
    
* **total_charges**: monto total que el cliente ha pagado en la suscripción.

In [33]:
# MOSTRAMOS INFORMACION GENERAL 

contract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        7043 non-null   object 
 1   begin_date         7043 non-null   object 
 2   end_date           7043 non-null   object 
 3   type               7043 non-null   object 
 4   paperless_billing  7043 non-null   object 
 5   payment_method     7043 non-null   object 
 6   monthly_charges    7043 non-null   float64
 7   total_charges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB


**Nota**: 

* La columna total_charges específica información en dólares, debido que es la facturación cobrada a cada usuario, se procederá a cambiar su tipo de 'object' a 'float'.

* Las columnas begin_date y end_date son de tipo object, mostrando información en fechas, se procederá a cambiarlas a tipo datetime.

In [34]:
# MODIFICAMOS EL TIPO DE LA COLUMNA 'total_charges'

contract['total_charges'] = pd.to_numeric(contract['total_charges'], errors = 'coerce')

**Nota**: en un principio, cuando total_charges era object, la columna no presentaba valores ausentes. Pues, cuando se cambio a tipo float, presenta valores ausentes, estos se trataran mas adelante.

In [35]:
# MOSTRAMOS LOS VALORES AUSENTES

contract.isna().sum()

customer_id           0
begin_date            0
end_date              0
type                  0
paperless_billing     0
payment_method        0
monthly_charges       0
total_charges        11
dtype: int64

In [36]:
# MOSTRAMOS INFORMACIÓN ESTADÍSTICA

contract.describe()

,monthly_charges,total_charges
count,7043.000000,7032.000000
mean,64.761692,2283.300441
std,30.090047,2266.771362
min,18.250000,18.800000
25%,35.500000,401.450000
50%,70.350000,1397.475000
75%,89.850000,3794.737500
max,118.750000,8684.800000


#### Conclusiones Intermedias de 'contract'

* El promedio de la mensualidad de los clientes es de 64.76 dólares.
* El promedio que la facturación pagada de los clientes es de 2283.30 dólares.
* La factura más baja es de 18.25 dólares mensual y pagada es de 18.80 dólares, lo cual puede traducirse a que existen clientes que solo abonaron un mes.
* La columna 'total_charges' era de tipo 'objet' y se cambio a tipo 'float'.
* En la columna 'total_charges' tenemos valores ausentes, con un total de 11 después del cambio de tipo.

### 1.2 Información General de 'internet'

In [40]:
# MOSTRAMOS UNA VISTA PREVIA

internet.sample(5)

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
2539,3629-ZNKXA,Fiber optic,No,Yes,Yes,No,Yes,Yes
3497,2665-NPTGL,Fiber optic,No,No,Yes,No,Yes,Yes
2468,9099-FTUHS,DSL,Yes,No,No,No,No,No
99,9445-ZUEQE,Fiber optic,No,Yes,No,Yes,No,No
4088,6261-LHRTG,DSL,No,Yes,Yes,No,No,No


In [41]:
# MODIFICAMOS LOS NOMBRES DE LAS COLUMNAS

internet_nombres = {
    'customerID': 'customer_id',
    'InternetService': 'internet_service',
    'OnlineSecurity': 'online_security',
    'OnlineBackup': 'online_backup',
    'DeviceProtection': 'device_protection',
    'TechSupport': 'tech_support',
    'StreamingTV': 'streaming_tv',
    'StreamingMovies': 'streaming_movies'
}

internet.rename(columns = internet_nombres, inplace = True)

internet.head()

,customer_id,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No


Tenemos la siguiente información:

**customer_id**: código único para cada cliente que se suscribio al paquete de Internet.

**internet_service**: tipo de Internet, DSL o fibra óptica.

**online_security**: si utiliza bloqueador de sitios web.

**online_backup**: si utiliza almacenamiento en la nube.

**device_protection**: si utiliza antivirus.

**tech_support**: si utiliza asistencia técnica dedicada.

**streaming_tv**: si contrato el servicio de streaming para TV.

**streaming_movies**: si contrato el servicio de ver películas.

In [42]:
# MOSTRAMOS INFORMACION GENERAL 

internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   customer_id        5517 non-null   object
 1   internet_service   5517 non-null   object
 2   online_security    5517 non-null   object
 3   online_backup      5517 non-null   object
 4   device_protection  5517 non-null   object
 5   tech_support       5517 non-null   object
 6   streaming_tv       5517 non-null   object
 7   streaming_movies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB


In [43]:
# MOSTRAMOS INFORMACIÓN ESTADÍSTICA

internet.describe()

,customer_id,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies
count,5517,5517,5517,5517,5517,5517,5517,5517
unique,5517,2,2,2,2,2,2,2
top,7590-VHVEG,Fiber optic,No,No,No,No,No,No
freq,1,3096,3498,3088,3095,3473,2810,2785


#### Conclusiones Intermedias de 'internet'

* Podemos observar que todos los tipos de las columnas son object.

* 6 de 8 columnas son de tendencia booleana, es decir, si o no. Podemos decir también que internet service presenta solo dos tipos: DSL y fibra óptica.

* No presenta valores ausentes.

### 1.3 Información General de 'personal'

In [46]:
# MOSTRAMOS UNA VISTA PREVIA

personal.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents
4072,2688-BHGOG,Male,1,No,No
2947,5131-PONJI,Male,0,Yes,Yes
3014,5857-TYBCJ,Male,1,Yes,No
5890,0383-CLDDA,Female,0,No,No
4083,0324-BRPCJ,Female,1,Yes,No


In [47]:
# MODIFICAMOS LOS NOMBRES DE LAS COLUMNAS

personal_nombres = {
    'customerID': 'customer_id',
    'SeniorCitizen': 'senior_citizen',
    'Partner': 'partner',
    'Dependents': 'dependents',
}

personal.rename(columns = personal_nombres, inplace = True)

personal.head()

,customer_id,gender,senior_citizen,partner,dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


Tenemos la siguiente información:
    
* **customer_id**: código único para cada cliente.

* **gender**: sexo de cada cliente.
    
* **senior_citizen**: si el cliente está jubilado o no.
    
* **partner**: si el cliente tiene pareja.
    
* **dependents**: si el cliente tiene personas a su cargo.

In [49]:
# MOSTRAMOS INFORMACION GENERAL 

personal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     7043 non-null   object
 1   gender          7043 non-null   object
 2   senior_citizen  7043 non-null   int64 
 3   partner         7043 non-null   object
 4   dependents      7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB


In [50]:
# MOSTRAMOS INFORMACIÓN ESTADÍSTICA

personal.describe()

,senior_citizen
count,7043.000000
mean,0.162147
std,0.368612
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


#### Conclusiones Intermedias de 'personal'

* Todas las columnas son de tipo object menos senior_citizen, que es int64.

* Es un dataset que muestra información categórica, como el sexo del cliente, si es jubilado o si dependen personas de él.

### 1.4 Información General de 'phone'

In [53]:
# MOSTRAMOS UNA VISTA PREVIA

phone.sample(5)

,customerID,MultipleLines
5017,9644-KVCNC,Yes
1729,7776-QWNFX,No
1915,4137-BTIKL,Yes
126,1095-WGNGG,Yes
6015,2164-SOQXL,No


In [54]:
# MODIFICAMOS LOS NOMBRES DE LAS COLUMNAS

phone_nombres = {
    'customerID': 'customer_id',
    'MultipleLines': 'multiple_lines',
}

phone.rename(columns = phone_nombres, inplace = True)

phone.head()

,customer_id,multiple_lines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes


Tenemos la siguiente información:
    
* **customer_id**: código único para cada cliente que se suscribió al paquete de telefonía.

* **multiple_lines**: si el cliente tiene una o más líneas telefónicas.

In [56]:
# MOSTRAMOS INFORMACION GENERAL 

phone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     6361 non-null   object
 1   multiple_lines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB


In [57]:
# MOSTRAMOS INFORMACIÓN ESTADÍSTICA

phone.describe()

,customer_id,multiple_lines
count,6361,6361
unique,6361,2
top,5575-GNVDE,No
freq,1,3390


#### Conclusiones Intermedias de 'phone'

* De igual manera con las columnas internet y personal, es una dataframe que muestra información categórica dándonos dos columnas de tipo objeto.

## 2. Preparación de los Datos

In [61]:
# MOSTRAMOS UNA MUESTRA DEL 'contract'

contract.sample(5)

,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges
6092,4024-CSNBY,2014-02-01,No,Two year,Yes,Bank transfer (automatic),94.25,6849.75
4996,3557-HTYWR,2016-03-01,No,Two year,No,Mailed check,74.05,3496.30
4323,3707-LRWZD,2017-02-01,2019-10-01 00:00:00,One year,No,Electronic check,84.05,2781.85
3405,0744-GKNGE,2014-10-01,No,Month-to-month,Yes,Bank transfer (automatic),86.80,5327.25
274,5940-AHUHD,2019-12-01,2020-01-01 00:00:00,Month-to-month,No,Mailed check,70.60,70.60


NOTA:

* Cabe destacar que en la columna 'end_date' la tenemos todavía como tipo object. ¿Por qué? Si la cambiamos a tipo datetime, no podríamos hacer lo siguiente:

    * Crearemos una nueva columna en 'contract' donde muestre la información si el cliente finalizó si o no la contratación de servicios con la empresa. La nueva columna, llamada 'fin_contrato' mostrará un 0 si el cliente no ha finalizado el contrato y 1 si ya lo finalizo.
    * Mas adelante, cambiaremos los tipos de las columnas 'begin_date' y 'end_date' a datetime, recordando que eran de tipo object.
    * Crearemos una nueva columna, llamada 'mes_suscript' que será el resultado de la resta entre la fecha entrante y de finalización de contrato, los cual nos data el total de meses de uso del servicio por cada cliente. Asimismo, tomando en cuenta la información del enunciado que es: " La información del contrato es válida a partir del 1 de febrero de 2020", utilizaremos ese dato para rellenar los valores ausentes, el dato es la resta entre esa fecha y la fecha inicial.

In [63]:
# DETERMINAMOS LA INFORMACIÓN DE LA COLUMNA 'fin_contrato'

contract['fin_contrato'] = (contract['end_date'] != 'No').astype('int')

contract.head(10)


,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,fin_contrato
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.50,0
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,1
5,9305-CDSKC,2019-03-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,99.65,820.50,1
6,1452-KIOVK,2018-04-01,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.40,0
7,6713-OKOMC,2019-04-01,No,Month-to-month,No,Mailed check,29.75,301.90,0
8,7892-POOKP,2017-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,104.80,3046.05,1
9,6388-TABGU,2014-12-01,No,One year,No,Bank transfer (automatic),56.15,3487.95,0


In [64]:
# PROCEDEMOS A REALIZAR LOS CAMBIOS DE TIPO EN LAS COLUMNAS 'begin_date' y 'end_date'

contract['begin_date'] = pd.to_datetime(contract['begin_date'], errors = 'coerce')

contract['end_date'] = pd.to_datetime(contract['end_date'], errors = 'coerce')

C:\Users\Admin\AppData\Local\Temp\ipykernel_6388\2350597061.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  contract['end_date'] = pd.to_datetime(contract['end_date'], errors = 'coerce')


In [65]:
# PROCEDEMOS A CREAR LA COLUMNA 'mes_suscript'

contract['mes_suscript'] = np.floor((contract.end_date - contract.begin_date)/np.timedelta64(1, 'M'))
contract['mes_suscript'] = contract['mes_suscript'].fillna(np.floor((pd.to_datetime('2020-02-01') - contract.begin_date)/np.timedelta64(1, 'M')))
contract.head()

,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,fin_contrato,mes_suscript
0,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,0,1.0
1,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,0,33.0
2,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,1,1.0
3,7795-CFOCW,2016-05-01,NaT,One year,No,Bank transfer (automatic),42.30,1840.75,0,44.0
4,9237-HQITU,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,1,1.0


In [66]:
# VEMOS A CLIENTES QUE YA HAN FINALIZADO SU CONTRADO

contract[contract['fin_contrato'] == 1]

,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,fin_contrato,mes_suscript
2,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,1,1.0
4,9237-HQITU,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,1,1.0
5,9305-CDSKC,2019-03-01,2019-11-01,Month-to-month,Yes,Electronic check,99.65,820.50,1,7.0
8,7892-POOKP,2017-07-01,2019-11-01,Month-to-month,Yes,Electronic check,104.80,3046.05,1,27.0
13,0280-XJGEX,2015-09-01,2019-10-01,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.30,1,48.0
...,...,...,...,...,...,...,...,...,...,...
7021,1699-HPSBG,2018-11-01,2019-11-01,One year,Yes,Electronic check,59.80,727.80,1,11.0
7026,8775-CEBBJ,2019-02-01,2019-11-01,Month-to-month,Yes,Bank transfer (automatic),44.20,403.35,1,8.0
7032,6894-LFHLY,2019-10-01,2019-11-01,Month-to-month,Yes,Electronic check,75.75,75.75,1,1.0
7034,0639-TSIQW,2014-03-01,2019-10-01,Month-to-month,Yes,Credit card (automatic),102.95,6886.25,1,65.0


### 2.1 Fusionamos el Dataframe

In [68]:
# APLICAMOS 'merge' PARA CREAR UN NUEVO DATAFRAME COMPLETO

proyecto = [contract, internet, personal, phone]

df = reduce(lambda  left,right: pd.merge(left,right,on=['customer_id'],
                                            how='outer'), proyecto)

df.head()

,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,fin_contrato,mes_suscript,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines
0,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,0,1.0,...,Yes,No,No,No,No,Female,0,Yes,No,NaN
1,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,0,33.0,...,No,Yes,No,No,No,Male,0,No,No,No
2,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,1,1.0,...,Yes,No,No,No,No,Male,0,No,No,No
3,7795-CFOCW,2016-05-01,NaT,One year,No,Bank transfer (automatic),42.30,1840.75,0,44.0,...,No,Yes,Yes,No,No,Male,0,No,No,NaN
4,9237-HQITU,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,1,1.0,...,No,No,No,No,No,Female,0,No,No,No


In [69]:
# MOSTRAMOS INFORMACIÓN GENERAL DEL DATAFRAME

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   customer_id        7043 non-null   object        
 1   begin_date         7043 non-null   datetime64[ns]
 2   end_date           1869 non-null   datetime64[ns]
 3   type               7043 non-null   object        
 4   paperless_billing  7043 non-null   object        
 5   payment_method     7043 non-null   object        
 6   monthly_charges    7043 non-null   float64       
 7   total_charges      7032 non-null   float64       
 8   fin_contrato       7043 non-null   int32         
 9   mes_suscript       7043 non-null   float64       
 10  internet_service   5517 non-null   object        
 11  online_security    5517 non-null   object        
 12  online_backup      5517 non-null   object        
 13  device_protection  5517 non-null   object        
 14  tech_sup

**Nota**:
    
* Creamos un nuevo dataframe con todos la información para efectos de trabajar con mayor comodidad, donde especifica toda la información de cada cliente, pero nos enfrentamos a valores aunsentes, los cuales se resolverán a continuación.

### 2.2 Tratamiento de Valores Ausentes

In [70]:
# MOSTRAMOS TODOS LOS VALORES AUSENTES

df.stb.missing()

,missing,total,percent
end_date,5174,7043,73.463013
online_security,1526,7043,21.666903
internet_service,1526,7043,21.666903
streaming_movies,1526,7043,21.666903
streaming_tv,1526,7043,21.666903
tech_support,1526,7043,21.666903
device_protection,1526,7043,21.666903
online_backup,1526,7043,21.666903
multiple_lines,682,7043,9.683374
total_charges,11,7043,0.156183


In [71]:
# MOSTRAMOS SI EXISTEN VALORES DUPLICADOS

df.duplicated().sum()

0

In [72]:
# RELLENAMOS LOS VALORES AUSENTES

columnas_faltantes = ['online_security', 
             'internet_service', 
             'streaming_movies', 
             'streaming_tv', 
             'tech_support', 
             'device_protection',
             'online_backup',
             'multiple_lines']

for col in columnas_faltantes:
    df[col] = df[col].fillna('sin_registrar')

df.stb.missing()

,missing,total,percent
end_date,5174,7043,73.463013
total_charges,11,7043,0.156183
customer_id,0,7043,0.000000
online_backup,0,7043,0.000000
dependents,0,7043,0.000000
partner,0,7043,0.000000
senior_citizen,0,7043,0.000000
gender,0,7043,0.000000
streaming_movies,0,7043,0.000000
streaming_tv,0,7043,0.000000


**NOTA**: 
    
* Decidimos que, para efectos de esa información que es categórica, se especifique si no la hay, como 'sin_registrar'.

In [74]:
# OBSERVAMOS LAS FILAS DONDE EXISTEN VALORES AUSENTES EN LA COLUMNA 'total_charges', SIENDO 11 FILAS.

df.loc[df['total_charges'].isna()]

,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,fin_contrato,mes_suscript,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines
488,4472-LVYGI,2020-02-01,NaT,Two year,Yes,Bank transfer (automatic),52.55,NaN,0,0.0,...,No,Yes,Yes,Yes,No,Female,0,Yes,Yes,sin_registrar
753,3115-CZMZD,2020-02-01,NaT,Two year,No,Mailed check,20.25,NaN,0,0.0,...,sin_registrar,sin_registrar,sin_registrar,sin_registrar,sin_registrar,Male,0,No,Yes,No
936,5709-LVOEQ,2020-02-01,NaT,Two year,No,Mailed check,80.85,NaN,0,0.0,...,Yes,Yes,No,Yes,Yes,Female,0,Yes,Yes,No
1082,4367-NUYAO,2020-02-01,NaT,Two year,No,Mailed check,25.75,NaN,0,0.0,...,sin_registrar,sin_registrar,sin_registrar,sin_registrar,sin_registrar,Male,0,Yes,Yes,Yes
1340,1371-DWPAZ,2020-02-01,NaT,Two year,No,Credit card (automatic),56.05,NaN,0,0.0,...,Yes,Yes,Yes,Yes,No,Female,0,Yes,Yes,sin_registrar
3331,7644-OMVMY,2020-02-01,NaT,Two year,No,Mailed check,19.85,NaN,0,0.0,...,sin_registrar,sin_registrar,sin_registrar,sin_registrar,sin_registrar,Male,0,Yes,Yes,No
3826,3213-VVOLG,2020-02-01,NaT,Two year,No,Mailed check,25.35,NaN,0,0.0,...,sin_registrar,sin_registrar,sin_registrar,sin_registrar,sin_registrar,Male,0,Yes,Yes,Yes
4380,2520-SGTTA,2020-02-01,NaT,Two year,No,Mailed check,20.00,NaN,0,0.0,...,sin_registrar,sin_registrar,sin_registrar,sin_registrar,sin_registrar,Female,0,Yes,Yes,No
5218,2923-ARZLG,2020-02-01,NaT,One year,Yes,Mailed check,19.70,NaN,0,0.0,...,sin_registrar,sin_registrar,sin_registrar,sin_registrar,sin_registrar,Male,0,Yes,Yes,No
6670,4075-WKNIU,2020-02-01,NaT,Two year,No,Mailed check,73.35,NaN,0,0.0,...,Yes,Yes,Yes,Yes,No,Female,0,Yes,Yes,Yes


In [75]:
# FILTRAMOS A LOS CLIENTES QUE SOLO SE SUSCRIBIERON 1 MES.

df.loc[df['mes_suscript'] == 1]

,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,fin_contrato,mes_suscript,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines
0,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,0,1.0,...,Yes,No,No,No,No,Female,0,Yes,No,sin_registrar
2,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,1,1.0,...,Yes,No,No,No,No,Male,0,No,No,No
4,9237-HQITU,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,1,1.0,...,No,No,No,No,No,Female,0,No,No,No
33,7310-EGVHZ,2020-01-01,NaT,Month-to-month,No,Bank transfer (automatic),20.20,20.20,0,1.0,...,sin_registrar,sin_registrar,sin_registrar,sin_registrar,sin_registrar,Male,0,No,No,No
34,3413-BMNZE,2020-01-01,NaT,Month-to-month,No,Bank transfer (automatic),45.25,45.25,0,1.0,...,No,No,No,No,No,Male,1,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6943,0032-PGELS,2019-10-01,2019-11-01,Month-to-month,No,Bank transfer (automatic),30.50,30.50,1,1.0,...,No,No,No,No,No,Female,0,Yes,Yes,sin_registrar
6979,5351-QESIO,2020-01-01,NaT,Month-to-month,No,Mailed check,24.20,24.20,0,1.0,...,No,No,No,No,No,Male,0,No,Yes,sin_registrar
7016,1471-GIQKQ,2020-01-01,NaT,Month-to-month,No,Electronic check,49.95,49.95,0,1.0,...,Yes,No,No,No,No,Female,0,No,No,No
7018,1122-JWTJW,2019-10-01,2019-11-01,Month-to-month,Yes,Mailed check,70.65,70.65,1,1.0,...,No,No,No,No,No,Male,0,Yes,Yes,No


In [76]:
# RELLENAMOS LA COLUMNA 'total_charges' CON LOS VALORES DE 'monthly_charges'

df['total_charges'].fillna(df[df['total_charges'].isna()]['monthly_charges'], inplace=True)  

df['total_charges'].isna().sum()

0

In [77]:
# REEMPLAZAMOS EL 0 EN 'mes_suscript' POR 1 Y CAMBIAMOS EL TIPO DE COLUMNA

df['mes_suscript'] = df['mes_suscript'].replace([0], 1).astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   customer_id        7043 non-null   object        
 1   begin_date         7043 non-null   datetime64[ns]
 2   end_date           1869 non-null   datetime64[ns]
 3   type               7043 non-null   object        
 4   paperless_billing  7043 non-null   object        
 5   payment_method     7043 non-null   object        
 6   monthly_charges    7043 non-null   float64       
 7   total_charges      7043 non-null   float64       
 8   fin_contrato       7043 non-null   int32         
 9   mes_suscript       7043 non-null   int32         
 10  internet_service   7043 non-null   object        
 11  online_security    7043 non-null   object        
 12  online_backup      7043 non-null   object        
 13  device_protection  7043 non-null   object        
 14  tech_sup

**Conclusiones Intermedias**:

* Se procedió a rellenar los valores ausentes de la columna 'total_charges' con los valores de la facturación del primer mes, porque se supone o se piensa, que esta en el mes en curso y por eso aun en sistema no se ha cargado la factura. 
    
* Si se puede ver que hay valores ausentes en la columna 'end_date', significan que esos clientes aun no han finalizado contrato, es decir, no tienen fecha de culminación.

* Se cambió el tipo de la columna mes_suscript a int64.

* Se determinó un dataframe completo y no varios para efectos de trabajar con la aplicación de modelos y gráficas.

## 3. Lista de Preguntas

1. ¿Qué tan propensos son los jubilados a finalizar contrato?

2. ¿Qué clientes son más propensos a finalizar contrato? ¿Los que tienen contrato mensual o anual?

3. ¿Cuál método de pago es el que presenta mayor índice de finalización de contrato?

4. ¿Cuál es el servicio de Internet que presenta más finalización de contratos?

5. Los servicios que ofrece la empresa como soporte técnico y protección de equipos, ¿cuáles clientes tienen más finalización de contrato, los que los usan o no?

6. Los clientes que poseen personas a cargo, ¿qué tendencia hay entre los que sí los tiene y los que no al finalizar contrato?

## 4. Plan Aproximado para la Resolución de la Tarea

1. Examinar con detenimiento que información del conjunto de datos se trabajará, específicar que columnas usaremos y cuales se desecharán. Asimismo, se decidirá cual sera la columna base para la división del conjunto de datos.

2. Aplicar modelos de Machine Learning, donde se dividan el conjunto de datos en un 80:20 para el conjunto de entrenamiento y prueba. Posteriormente se aplicaran procesos de preparación de modelos, como serán las técnicas One Hot EnCoding y el Scaling y Upsampling, si amerita el caso.

3. ¿Cuáles modelos se aplicarán? Pues, en este apartado, decidimos aplicar Regresión Logística, Arbol de Decisión, Bosque Aleatorio, CatBoost, XGBoost y las Redes Neuronales. Pensamos que con seis modelos podemos descrifrar buenos margenes en la puntuación ROC-AUC, que tiene que ser mayor al 85%.

4. De último, se concluirá mostrando y comparando los resultados arrojados por los distintos modelos aplicados, se determinará cual es el mejor modelo y se redactará una informe expresando los resultados y las recomendaciones a la empresa, para efectos de que la gerencia tome decisiones en pro de evitar más fuga de clientes.